<a href="https://colab.research.google.com/github/FlyAIBox/AIAgent101/blob/main/06-agent-evaluation/01_langfuse_integration_openai_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI SDK集成Langfuse获得完整的可观测性
---
description: 只需替换 import 语句，就能用 Langfuse 版本的 OpenAI SDK 获得完整的可观测性
category: Integrations
---


# 示例手册：OpenAI 集成（Python）


这是一个示例手册，演示如何在 Python 项目中集成 Langfuse 与 OpenAI。

Langfuse 会记录每次模型调用的输入输出，帮助你排查问题并评估质量。

按照 [集成指南](https://langfuse.com/integrations/model-providers/openai-py) 将本集成添加到你的 OpenAI 项目中。


## 环境准备


该集成适用于 OpenAI SDK `>=0.27.8`；若要使用异步函数和流式输出，请确保 SDK 版本 `>=1.0.0`。


In [1]:
%pip install langfuse==3.3.0 openai==1.107.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.36.0
    Uninstalling opentelemetry-api-1.36.0:
      Successfully uninstalled opentelemetry-api-1.36.0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.57b0
    Uninstalling opentelemetry-semantic-conventions-0.57b0:
      Successfully uninstalled opentelemetry-semantic-conventions-0.57b0
  Attempting uninstall: opentelemetry-sdk
    Found existing installation: opentelemetry-sdk 1.36.0
    Uninstalling opentelemetry-sdk-1

In [3]:
# 环境变量配置
# 设置OpenAI API密钥，这是使用OpenAI模型所必需的
import os, getpass

def _set_env(var: str):
    """
    安全地设置环境变量
    如果环境变量不存在，会提示用户输入
    """
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 设置OpenAI API密钥
# 您需要从 https://platform.openai.com/api-keys 获取API密钥
_set_env("OPENAI_API_KEY")
# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")


# 在项目设置页获取密钥：https://cloud.langfuse.com,
# os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-2cf6d992-461d-4b3c-a47e-96285c1e5526@FLYAIBOX"
# os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-55c2fa56-c913-44be-916b-85ba498dcb8a@FLYAIBOX"
# PUBLIC KEY
_set_env("LANGFUSE_PUBLIC_KEY")
# SECRET KEY
_set_env("LANGFUSE_SECRET_KEY")
# 🇪🇺 欧盟区域(推荐) https://cloud.langfuse.com
# 🇺🇸 美国区域 https://us.cloud.langfuse.com
_set_env("LANGFUSE_HOST")

OPENAI_API_KEY: ··········
OPENAI_BASE_URL: ··········
LANGFUSE_PUBLIC_KEY: ··········
LANGFUSE_SECRET_KEY: ··········
LANGFUSE_HOST: ··········


In [4]:
# Langfuse 提供了对原生 OpenAI SDK 的封装，接口保持一致但会自动上报调用数据
# 只需替换原本的 `import openai`，其余代码可以保持不变
from langfuse.openai import openai


## 示例

### 文本聊天补全


In [5]:
# 示例：发起一次标准的聊天补全请求
# Langfuse 会记录请求名称 (name)、模型 (model)、消息内容 (messages) 以及自定义元数据 (metadata)
completion = openai.chat.completions.create(
  name="test-chat",  # 追踪名称，可在 Langfuse 中快速定位这次调用
  model="gpt-4o",  # 指定使用的模型
  messages=[
      {"role": "system", "content": "您是一个非常精确的计算器。您只输出计算结果。"},  # system 消息：定义助手的身份/行为
      {"role": "user", "content": "1 + 1 = "}],  # user 消息：用户的真实输入
  temperature=0,  # 温度越低越稳定，适合需要确定答案的场景
  metadata={"someMetadataKey": "someValue"},  # 自定义元数据，会显示在 Langfuse 调试界面
)

# 如果需要查看模型响应，可使用 completion.choices[0].message.content


### 图像聊天补全

这是一个演示 OpenAI 视觉能力的简单示例。你可以在 `user` 消息中直接传入图片。


In [6]:
# 示例：向模型发送图像输入
completion = openai.chat.completions.create(
  name="test-url-image",  # 追踪名称
  model="gpt-4o-mini",  # GPT-4o、GPT-4o mini、GPT-4 Turbo 均具备视觉能力
  messages=[
      {"role": "system", "content": "您是一个被训练来描述和解释图像的AI。描述图像中的主要物体和动作。"},  # 指定模型任务
      {"role": "user", "content": [
        {"type": "text", "text": "这幅画描绘了什么？"},  # 文字提示
        {
          "type": "image_url",
          "image_url": {
            "url": "https://static.langfuse.com/langfuse-dev/langfuse-example-image.jpeg",  # 示例图片 URL，Langfuse 会记录该链接
          },
        },
      ],
    }
  ],
  temperature=0,  # 图像描述多为事实陈述，建议设置较低温度
  metadata={"someMetadataKey": "someValue"},  # 自定义元数据
)

# Langfuse 会将图片 URL 也记录在追踪详情中，方便复现问题


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看生成记录。

![聊天补全截图](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509211139659.png)


### 流式聊天补全

这是一个演示 OpenAI 流式输出能力的简单示例。


In [8]:
# 示例：开启流式输出，实时获取模型返回的 Token
completion = openai.chat.completions.create(
  name="test-chat",  # 追踪名称
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "您是一位专业的喜剧演员。"},
      {"role": "user", "content": "讲一个笑话给我听。"}],
  temperature=0,  # 温度越高，笑话越发散；此处设为 0 方便演示
  metadata={"someMetadataKey": "someValue"},
  stream=True  # 打开流式模式后，API 会边生成边返回
)

# completion 变为一个生成器，依次产出增量内容；打印时记得取消换行
for chunk in completion:
  print(chunk.choices[0].delta.content, end="")


当然！你知道为什么数学书总是很忧郁吗？因为它有太多的问题！None

### 异步聊天补全

该示例使用 OpenAI 的异步客户端。Langfuse 配置可通过环境变量或 `openai` 模块上的属性传入。


In [9]:
# 异步示例：Langfuse 同样兼容 OpenAI 的 Async 客户端
from langfuse.openai import AsyncOpenAI

async_client = AsyncOpenAI()  # 实例化异步客户端，将自动复用环境变量中的 Langfuse 配置


In [10]:
# 在异步函数内调用聊天补全接口，需要使用 await 等待结果
completion = await async_client.chat.completions.create(
  name="test-chat",  # 为本次调用命名
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "你是一个非常精确的计算器。你只输出计算结果。"},
      {"role": "user", "content": "1 + 100 = "}],
  temperature=0,
  metadata={"someMetadataKey": "someValue"},
)

# Langfuse 会自动关联异步调用上下文，不需要额外配置


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看生成记录。


### 函数调用

该示例演示如何借助 Pydantic 构建函数模式。


In [13]:
%pip install pydantic==2.11.9

In [14]:
from typing import List
from pydantic import BaseModel

# 定义函数调用返回值的数据结构，让模型生成结构化的 JSON
class StepByStepAIResponse(BaseModel):
    title: str  # 标题：例如“装机步骤”
    steps: List[str]  # 步骤列表：每个元素是一句描述

schema = StepByStepAIResponse.schema()  # 返回 JSON Schema，供 OpenAI 函数调用使用


/tmp/ipython-input-414262564.py:9: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  schema = StepByStepAIResponse.schema()  # 返回 JSON Schema，供 OpenAI 函数调用使用


In [ ]:
import json

# 示例：引导模型调用我们定义的函数，并返回结构化结果
response = openai.chat.completions.create(
    name="test-function",
    model="gpt-4o-0613",  # 支持函数调用的模型版本
    messages=[
       {"role": "user", "content": "如何组装一台电脑"}
    ],
    functions=[
        {
          "name": "get_answer_for_user_query",  # 函数名称，需要与业务代码保持一致
          "description": "分步骤为用户提供答案",  # 告诉模型函数的用途
          "parameters": StepByStepAIResponse.schema()  # Pydantic 自动生成的参数定义
        }
    ],
    function_call={"name": "get_answer_for_user_query"}  # 强制模型调用指定函数
)

# Langfuse 会记录函数调用的入参与出参，便于追踪
output = json.loads(response.choices[0].message.function_call.arguments)  # 将字符串反序列化为 Python 字典


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看生成记录。

示例：https://cloud.langfuse.com/project/cmequpe0j00euad07w6wrvkzg/traces/8942d39f62095985bf891156bbd563b9?timestamp=2025-09-21T03:42:03.721Z



## Langfuse 功能（用户、标签、元数据、会话）

你可以在 OpenAI 请求中加入额外属性，以启用更多 Langfuse 功能。Langfuse 集成会自动解析这些字段。完整功能列表见 [文档](https://langfuse.com/integrations/model-providers/openai-py#custom-trace-properties)。


In [19]:
result = openai.chat.completions.create(
    name="test-chat-with-attributes",  # trace 名称，对应 Langfuse 中的 Trace.name
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "您是一个非常精确的计算器。您只输出计算结果。"},
        {"role": "user", "content": "1 + 1 = "}],
    temperature=0,
    metadata={
        "langfuse_session_id": "session_123", # 会话 ID，用于区分不同对话/请求
        "langfuse_user_id": "user_456", # 业务用户 ID，让你在 Langfuse 中按用户聚合
        "langfuse_tags": ["calculator"], # trace 标签，可用于 Langfuse 控制台筛选
        "someMetadataKey": "someValue"  # trace 元数据，适合记录业务上下文
    }
)


示例追踪：


![image-20250921115506459](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509211155197.png)

## 将多次生成归并为单个 Trace

在实际应用中，往往需要多次调用 OpenAI。借助 `@observe()` 装饰器，可以把一次 API 调用中的所有 LLM 请求归入 Langfuse 中同一个 `trace`。


In [20]:
from langfuse.openai import openai
from langfuse import observe

# 【@observe 装饰器】会自动：
# 1. 为 main 函数创建一个顶层 trace
# 2. 捕获函数内部的所有 Langfuse/OpenAI 调用，并将它们串联为一个完整链路
@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main(country: str, user_id: str, **kwargs) -> str:
    # 嵌套调用 1：询问国家首都
    capital = openai.chat.completions.create(
      name="geography-teacher",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "您是一位地理老师，帮助学生学习国家的首都。当被问及时，只输出首都。"},
          {"role": "user", "content": country}],
      temperature=0,
    ).choices[0].message.content  # 读取模型回复

    # 嵌套调用 2：请模型写一首关于首都的诗
    poem = openai.chat.completions.create(
      name="poet",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "你是一位诗人。创作一首关于城市的诗。"},
          {"role": "user", "content": capital}],
      temperature=1,  # 提高温度，让诗歌更有创意
      max_tokens=200,  # 控制输出长度
    ).choices[0].message.content

    return poem

# 直接调用 main 函数，Langfuse 会自动生成 trace 并可在控制台查看链路
print(main("北京", "FLY"))


在华夏大地的北方一隅，   
长城以北，黄土以东，   
古老与现代在此共舞，   
北京，这座城市的心跳如钟。   

紫禁城巍峨，诉说着往昔的传奇，   
天安门广场，见证了历史的更替。   
在胡同深处，岁月的足迹依稀，   
四合院里，古老的韵味铭刻心底。   

繁华的街道，车流如水，   
霓虹灯下，城市的梦境无穷。   
摩天大楼直插云霄，   
孕育新的希望与光荣。   

文人翰墨，在此挥洒千载，   
创新创业，年轻的热血昂扬。   
北京，不息的脉动中，   
承载着中原儿女的


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看完整链路。

![多次 OpenAI 调用的追踪图](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509211159038.png)


## 完整方案：与 Langfuse SDK 协同

`trace` 是 Langfuse 的核心对象，你可以为它附加丰富的元数据。详见 [Python SDK 文档](https://langfuse.com/docs/sdk/python#traces-1)。

自定义 trace 后可以实现以下能力：
- 自定义名称，用来区分不同类型的链路
- 以用户为粒度的追踪
- 通过版本与发布信息进行实验管理
- 保存自定义元数据


In [21]:
from langfuse.openai import openai
from langfuse import observe, get_client

langfuse = get_client()  # 获取底层 Langfuse 客户端，可在装饰器之外手动操作 trace

@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main(country: str, user_id: str, **kwargs) -> str:
    # 嵌套调用 1：获取国家首都
    capital = openai.chat.completions.create(
      name="geography-teacher",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "您是一位地理老师，帮助学生学习国家的首都。当被问及时，只输出首都。"},
          {"role": "user", "content": country}],
      temperature=0,
    ).choices[0].message.content

    # 嵌套调用 2：根据首都生成诗歌
    poem = openai.chat.completions.create(
      name="poet",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "你是一位诗人。创作一首关于城市的诗。"},
          {"role": "user", "content": capital}],
      temperature=1,
      max_tokens=200,
    ).choices[0].message.content

    # 手动更新当前 trace 的属性，让仪表盘信息更完整
    langfuse.update_current_trace(
        name="City poem generator",  # 自定义 trace 名称
        session_id="1234",  # 业务会话 ID
        user_id=user_id,  # 业务用户 ID
        tags=["tag1", "tag2"],  # 标签，支持在 Langfuse 中搜索
        public=True,  # 是否允许分享 Trace 链接
        metadata = {"env": "development"}  # 自定义元数据，例如环境标记
    )

    return poem

# create_trace_id() 会生成一个可复用的追踪 ID，你也可以改为自己的业务 ID
trace_id = langfuse.create_trace_id()

# 通过关键字参数 `langfuse_observation_id` 将 trace_id 传递给装饰器，方便串联上下游调用
print(main("北京", "admin", langfuse_observation_id=trace_id))


在万里长城的臂弯间，  
北平古都今正繁忙。  
故宫琉璃辉映朝霞，  
长安街头人潮涌荡。  

曾闻胭脂车马喧闹，  
如今高楼接天如影。  
胡同深巷承载旧梦，  
CBD内霓虹交映。  

时光在四合院中流淌，  
京腔胡同诉说历史。  
香山红叶忆往事，  
东单西单展新貌。  

莲花池畔寻一丝宁静，  
天坛祈福迎来朝暮。  
在这古老而又现代的城，  
岁月如歌，铭刻永恒。  


示例：
![image-20250921120555385](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509211205931.png)

## 以编程方式添加评分


你可以向 trace 添加 [评分](https://langfuse.com/docs/scores)，记录用户反馈或自动化评估结果。评分可用于在 Langfuse 中筛选追踪，并会显示在控制台中。详见评分文档。

评分通过 `trace_id` 与对应的 trace 关联。


In [22]:
from langfuse import observe, get_client

langfuse = get_client()  # 获取底层客户端，用于主流程之外的操作

@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main():
    # 在装饰器内部，可随时获取当前 trace 的 ID
    trace_id = langfuse.get_current_trace_id()

    # TODO: 在此处编写你的业务逻辑，例如继续调用其他 API、处理用户输入等

    return "res", trace_id

# 执行被装饰的函数，Langfuse 会生成 trace
_, trace_id = main()

# 在 trace 上下文外部也可以继续操作，例如向这次 trace 添加评分
langfuse.create_score(
    trace_id=trace_id,  # 指定要打分的 trace
    name="my-score-name",  # 评分名称，用于区分不同指标
    value=1  # 分值，可以是布尔、整数、浮点数，视业务场景而定
)


示例：![image-20250921120901924](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509211209271.png)